In [1]:
!pip install python-mimeparse

In [3]:
import json
import pandas as pd
import email
from email import policy
from email.parser import BytesParser
import pandas as pd
import os

In [43]:
sample_df = df.sample(10)

In [44]:
sample_df.head()

,user_id,msg_id,sg_event_id,subject,mime,email_from,email_to,email_date,originating_ip,originating_ip_country,lang,s3_path
27,3956193,TkRqGkKYRruCLPJ2oxyjVA.recvd-97d7ddb5-s4hqs-1-...,cHJvY2Vzc2VkLTM5NTYxOTMtVGtScUdrS1lScnVDTFBKMm...,Welcome to Comfort Keepers Mid Atlantic,DKIM-Signature: v=1; a=rsa-sha256; c=relaxed/r...,"""Comfort Keepers Mid Atlantic"" <No-Reply@AppSu...",nathalyosorio1204@gmail.com,2025-03-18,54.146.18.169,United States,en,users/3956193/05df04f3-b0c5-4249-bc8c-fede78cf...
47,3956193,oLsR7MlcQ6qBvbQlmmxMlQ.recvd-7cf7589bf9-sr2dk-...,cHJvY2Vzc2VkLTM5NTYxOTMtb0xzUjdNbGNRNnFCdmJRbG...,Welcome to CK 581 MN,DKIM-Signature: v=1; a=rsa-sha256; c=relaxed/r...,"""Comfort Keepers 581 Minnesota"" <No-Reply@AppS...",marisa21marie@gmail.com,2025-03-21,100.24.253.107,United States,en,users/3956193/4b87f2c5-a2cf-48ee-a605-1de54637...
20,3956193,LEYZwM4mS2KD2-0LdzfEsw.recvd-578d769b87-qbqf5-...,cHJvY2Vzc2VkLTM5NTYxOTMtTEVZWndNNG1TMktEMi0wTG...,Self Care Bingo - completed,DKIM-Signature: v=1; a=rsa-sha256; c=relaxed/r...,"""Comfort Keepers Central NJ"" <No-Reply@AppSupp...",sheilatruncellito@comfortkeepers.com,2025-03-14,34.238.142.123,United States,en,users/3956193/3224046e-5a9d-46ba-a2a1-fe16180c...
9,3956193,CDOFFHs2QDqSWjdb0txoSQ.recvd-8575c86f9d-7xql8-...,cHJvY2Vzc2VkLTM5NTYxOTMtQ0RPRkZIczJRRHFTV2pkYj...,Self Care Bingo - completed,DKIM-Signature: v=1; a=rsa-sha256; c=relaxed/r...,"""Comfort Keepers Central NJ"" <No-Reply@AppSupp...",laurentabor@comfortkeepers.com,2025-03-15,35.153.130.251,United States,en,users/3956193/a1de6859-7ce4-483a-93d4-573e3ce8...
35,3956193,arxSIytLRKGM2n040lXX_g.recvd-7cf7589bf9-zbglx-...,cHJvY2Vzc2VkLTM5NTYxOTMtYXJ4U0l5dExSS0dNMm4wND...,App Invite - Right at Home of East Texas,DKIM-Signature: v=1; a=rsa-sha256; c=relaxed/r...,"""Right at Home of East Texas"" <No-Reply@AppSup...",pamjanjan@gmail.com,2025-03-26,3.228.25.63,United States,en,users/3956193/2b810a58-570f-400b-b91c-45b4f897...


In [56]:
raw_mime = sample_df.iloc[2]['mime']

In [69]:
sample_df = df.sample(5)

In [71]:
def extract_data_from_raw_mime(raw_mime):
    raw_mime = raw_mime.encode('utf-8')
    msg = BytesParser(policy=policy.default).parsebytes(raw_mime)
    
    from_ = msg['From']
    to = msg['To']
    subject = msg['Subject']
    date = msg['Date']
    content_type = msg.get_content_type()
    text_body = ""
    html_body = ""
    
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = part.get("Content-Disposition", "")
            if content_type == "text/plain" and "attachment" not in content_disposition:
                text_body = part.get_payload(decode=True).decode(part.get_content_charset('utf-8'))
            elif content_type == "text/html":
                html_body = part.get_payload(decode=True).decode(part.get_content_charset('utf-8'))
    else:
        content_type = msg.get_content_type()
        if content_type == "text/plain":
            text_body = msg.get_payload(decode=True).decode(msg.get_content_charset('utf-8'))
        elif content_type == "text/html":
            html_body = msg.get_payload(decode=True).decode(msg.get_content_charset('utf-8'))

    # Optional: Save HTML to a file
    if html_body:
        os.makedirs("html_output", exist_ok=True)
        filename = subject.lower().replace(" ", "_").replace("/", "_")[:50]  # safe filename
        with open(f"html_output/{filename}.html", "w", encoding='utf-8') as f:
            f.write(html_body)

    return {
        "from_address": from_,
        "to_address": to,
        "subject": subject,
        "Date": date,
        "Content-Type": content_type,
        "email_text": text_body
    }


In [78]:
extracted_df = df['mime'].apply(extract_data_from_raw_mime).apply(pd.Series)

df = pd.concat([df, extracted_df], axis=1)

In [82]:
df.to_csv("outputs/text_extracted_csv.csv",index=False)